In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv
/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv
/kaggle/input/rohlik-orders-forecasting-challenge/train.csv
/kaggle/input/rohlik-orders-forecasting-challenge/test.csv
/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv


In [94]:
train_data=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/train.csv")
test_data=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/test.csv")
solution_example=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv")
test_calenders_data=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv")
train_calenders_data=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv")

In [95]:
train_data.dtypes,test_data.dtypes,train_data.shape

(warehouse                  object
 date                       object
 orders                    float64
 holiday_name               object
 holiday                     int64
 shutdown                    int64
 mini_shutdown               int64
 shops_closed                int64
 winter_school_holidays      int64
 school_holidays             int64
 blackout                    int64
 mov_change                float64
 frankfurt_shutdown          int64
 precipitation             float64
 snow                      float64
 user_activity_1           float64
 user_activity_2           float64
 id                         object
 dtype: object,
 warehouse                 object
 date                      object
 holiday_name              object
 holiday                    int64
 shops_closed               int64
 winter_school_holidays     int64
 school_holidays            int64
 id                        object
 dtype: object,
 (7340, 18))

In [96]:
train_columns=list(test_data.columns)
train_columns.append("orders")
train_data=train_data[train_columns]


In [97]:
train_data['date']=pd.to_datetime(train_data['date'])

In [98]:
def add_additional_date_features(df):
    df["month"]=df.date.dt.month
    df["day_num_week"]=df.date.dt.weekday
    df["year"]=df.date.dt.year
    return df

In [99]:
def find_outliers_iqr(df, column, threshold=1.5):
    """
    Identifies outliers in a DataFrame column using the IQR method.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    column (str): The name of the column to check for outliers.
    threshold (float): The multiplication factor for the IQR.

    Returns:
    pd.DataFrame: A DataFrame containing the outliers.
    """

    # Calculate the IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - (threshold * IQR)
    upper_bound = Q3 + (threshold * IQR)

    # Identify outliers
    #outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return lower_bound,upper_bound 

In [100]:
def mark_outlier(df):
    lower_bound,upper_bound =find_outliers_iqr(df,"orders", threshold=1.5)
    df_outliers = df[(df["orders"] < lower_bound) | (df["orders"] > upper_bound)]
    df_non_outliers = df[(df["orders"] >= lower_bound) & (df["orders"] <=upper_bound)]
    df_outliers["IS_OUTLIER"]=1
    df_non_outliers["IS_OUTLIER"]=0
    df=pd.concat([df_outliers,df_non_outliers])
    return df

In [101]:
def transform_data(df):
    df= add_additional_date_features(df)
    encode_columns=['month','day_num_week','holiday_name']
    df=pd.get_dummies(df,columns=encode_columns)
    df=mark_outlier(df)
    return df
    
    

In [102]:
train_data=transform_data(train_data)

(7340, 52)


/tmp/ipykernel_33/2844901373.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outliers["IS_OUTLIER"]=1
/tmp/ipykernel_33/2844901373.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_outliers["IS_OUTLIER"]=0


In [104]:
train_data.dtypes

warehouse                                                                           object
date                                                                        datetime64[ns]
holiday                                                                              int64
shops_closed                                                                         int64
winter_school_holidays                                                               int64
school_holidays                                                                      int64
id                                                                                  object
orders                                                                             float64
year                                                                                 int32
month_1                                                                               bool
month_2                                                                               bool

In [105]:
train_data.to_csv("input_data.csv")